# Pckage List

In [5]:
import numpy as np, pandas as pd, warnings, os, openai, json
from tqdm.auto import tqdm
from openai import OpenAI
warnings.filterwarnings('ignore')
import requests


# Data Load

In [2]:
df = pd.read_csv("../data/prep1.csv", encoding='cp949')

# Function

## 위치 관련 정보 제공

In [6]:
def get_lat_lon(location_name):
    url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': location_name,
        'format': 'json'
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (compatible; ChatGPT-Example/1.0; +http://yourdomain.com)'
    }

    response = requests.get(url, params=params, headers=headers)

    # 응답 확인
    if response.status_code != 200:
        print(f"요청 실패: {response.status_code}")
        print("응답 내용:", response.text)
        return None

    try:
        data = response.json()
        if data:
            lat = data[0]['lat']
            lon = data[0]['lon']
            return float(lat), float(lon)
        else:
            print("위치 정보를 찾을 수 없습니다.")
            return None
    except ValueError as e:
        print("JSON 파싱 오류:", e)
        print("응답 내용:", response.text)
        return None

# 테스트
location = '진천군'
result = get_lat_lon(location)
print(f"{location} 위도/경도: {result}")


진천군 위도/경도: (36.855461, 127.4353927)


## API 키 설정

In [10]:
key = open('../../../api_key.txt','r')
api_key = key.read()
openai.api_key = api_key

FileNotFoundError: [Errno 2] No such file or directory: '../../../api_key.txt'

In [ ]:
client = OpenAI(api_key=api_key)
def invoke(question, info = user_info, model="gpt-4o", temperature=0.7):
    response = client.chat.completions.create(
        model=model,
        messages = [
                    {"role": "system", 
                        "content": f"""너는 사람들의 질문에 친절히 답해주는 도우미야.
                        너는 서울시에서 운영하는 대여 공구 찾기 정보 시스템이야. 
                        관련 정보는 {user_info}와 같아. 
                        이정보를 통해서 사용자의 답변에 친절해 답해줘.
                        """
                        },
                    {"role": "user", 
                    "content": question}
                ],
        temperature=temperature
    )
    return response.choices[0].message.content

In [ ]:
question = f"나는 고압세척기를 빌리고싶어. 내가 사는 곳은 황학동이야. 근처에 있는 대여 장소가 어디 있는지 알려줘"

In [ ]:
answer = invoke(question)
print("💬 GPT 응답:", answer)

In [ ]:
def invoke(question: str):
    prompt = f"""
    너는 도우미야 사용자의 물음에 친절히 답해줘야해.
    """
    response = client.responses.create(
        model='gpt-4.1',
        messages=[
                {"role": "system", "content": "당신은 완벽한 도우미입니다."},
                {"role": "user", "content": prompt}
            ],
        input = question
        )
    return response